# **How To Generate an Image**
1. Pick an ImageNet class, and find its number.
All class numbers can be found here: https://deeplearning.cms.waikato.ac.nz/user-guide/class-maps/IMAGENET/
2. Change the variable class_number accordingly in the code cell below.
3. Change the variable class_name accordingly in the code cell below.
4. Run all cells below in their order.
6. Wait for the code to stop running.

You can find the result in a folder named OUTPUTS/class_name , where class_name is replaced with the value of this variable.

In [5]:
# CHANGE THE VARIABLES BELOW
class_number = 0
class_name = "tench"

**Mount**

In [2]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

%cd '/content/drive/My Drive/workshop'

Mounted at /content/drive/
/content/drive/My Drive/workshop


**GPU**

In [3]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Oct  3 12:32:32 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

**Imports**


In [4]:
import sys
import numpy as np
import torch
import torch.optim
import time
from torch import nn
from torch.nn import functional as F
import torchvision.transforms.functional as TF
import torchvision.transforms as T
import random
from torchvision import transforms
import torchvision
from PIL import Image
import pandas as pd
import os

%pip install kornia
%pip install madgrad

import kornia
import kornia.augmentation as K
from madgrad import MADGRAD
from torchvision.models import inception_v3


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 705.7/705.7 kB 9.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for madgrad: filename=madgrad-1.3-py3-none-any.whl size=11867 sha256=9ffdb42b0a8d95bb2e7e27e9945d90b74d791bda04a3097ee34c0a8d2d74dcf0
  Stored in directory: /root/.cache/pip/wheels/d9/a3/83/7ed1ddc517cd87cad4e3a4aec7f8ea1d5e83a5ff282e51490a
Successfully built madgrad


**Deep Image Prior**

In [6]:
def add_module(self, module):
	self.add_module(str(len(self) + 1), module)


torch.nn.Module.add = add_module

# @title
class Downsampler(nn.Module):
	'''
		http://www.realitypixels.com/turk/computergraphics/ResamplingFilters.pdf
	'''

	def __init__(self, n_planes, factor, kernel_type, phase=0, kernel_width=None, support=None, sigma=None,
				 preserve_size=False):
		super(Downsampler, self).__init__()

		assert phase in [0, 0.5], 'phase should be 0 or 0.5'

		if kernel_type == 'lanczos2':
			support = 2
			kernel_width = 4 * factor + 1
			kernel_type_ = 'lanczos'

		elif kernel_type == 'lanczos3':
			support = 3
			kernel_width = 6 * factor + 1
			kernel_type_ = 'lanczos'

		elif kernel_type == 'gauss12':
			kernel_width = 7
			sigma = 1 / 2
			kernel_type_ = 'gauss'

		elif kernel_type == 'gauss1sq2':
			kernel_width = 9
			sigma = 1. / np.sqrt(2)
			kernel_type_ = 'gauss'

		elif kernel_type in ['lanczos', 'gauss', 'box']:
			kernel_type_ = kernel_type

		else:
			assert False, 'wrong name kernel'

		# note that `kernel width` will be different to actual size for phase = 1/2
		self.kernel = get_kernel(factor, kernel_type_, phase, kernel_width, support=support, sigma=sigma)

		downsampler = nn.Conv2d(n_planes, n_planes, kernel_size=self.kernel.shape, stride=factor, padding=0)
		downsampler.weight.data[:] = 0
		downsampler.bias.data[:] = 0

		kernel_torch = torch.from_numpy(self.kernel)
		for i in range(n_planes):
			downsampler.weight.data[i, i] = kernel_torch

		self.downsampler_ = downsampler

		if preserve_size:

			if self.kernel.shape[0] % 2 == 1:
				pad = int((self.kernel.shape[0] - 1) / 2.)
			else:
				pad = int((self.kernel.shape[0] - factor) / 2.)

			self.padding = nn.ReplicationPad2d(pad)

		self.preserve_size = preserve_size

	def forward(self, input):
		if self.preserve_size:
			x = self.padding(input)
		else:
			x = input
		self.x = x
		return self.downsampler_(x)


def get_kernel(factor, kernel_type, phase, kernel_width, support=None, sigma=None):
	assert kernel_type in ['lanczos', 'gauss', 'box']

	# factor  = float(factor)
	if phase == 0.5 and kernel_type != 'box':
		kernel = np.zeros([kernel_width - 1, kernel_width - 1])
	else:
		kernel = np.zeros([kernel_width, kernel_width])

	if kernel_type == 'box':
		assert phase == 0.5, 'Box filter is always half-phased'
		kernel[:] = 1. / (kernel_width * kernel_width)

	elif kernel_type == 'gauss':
		assert sigma, 'sigma is not specified'
		assert phase != 0.5, 'phase 1/2 for gauss not implemented'

		center = (kernel_width + 1.) / 2.
		print(center, kernel_width)
		sigma_sq = sigma * sigma

		for i in range(1, kernel.shape[0] + 1):
			for j in range(1, kernel.shape[1] + 1):
				di = (i - center) / 2.
				dj = (j - center) / 2.
				kernel[i - 1][j - 1] = np.exp(-(di * di + dj * dj) / (2 * sigma_sq))
				kernel[i - 1][j - 1] = kernel[i - 1][j - 1] / (2. * np.pi * sigma_sq)
	elif kernel_type == 'lanczos':
		assert support, 'support is not specified'
		center = (kernel_width + 1) / 2.

		for i in range(1, kernel.shape[0] + 1):
			for j in range(1, kernel.shape[1] + 1):

				if phase == 0.5:
					di = abs(i + 0.5 - center) / factor
					dj = abs(j + 0.5 - center) / factor
				else:
					di = abs(i - center) / factor
					dj = abs(j - center) / factor

				pi_sq = np.pi * np.pi

				val = 1
				if di != 0:
					val = val * support * np.sin(np.pi * di) * np.sin(np.pi * di / support)
					val = val / (np.pi * np.pi * di * di)

				if dj != 0:
					val = val * support * np.sin(np.pi * dj) * np.sin(np.pi * dj / support)
					val = val / (np.pi * np.pi * dj * dj)

				kernel[i - 1][j - 1] = val


	else:
		assert False, 'wrong method name'

	kernel /= kernel.sum()

	return kernel

class Concat(nn.Module):
	def __init__(self, dim, *args):
		super(Concat, self).__init__()
		self.dim = dim

		for idx, module in enumerate(args):
			self.add_module(str(idx), module)

	def forward(self, input):
		inputs = []
		for module in self._modules.values():
			inputs.append(module(input))

		inputs_shapes2 = [x.shape[2] for x in inputs]
		inputs_shapes3 = [x.shape[3] for x in inputs]

		if np.all(np.array(inputs_shapes2) == min(inputs_shapes2)) and np.all(
				np.array(inputs_shapes3) == min(inputs_shapes3)):
			inputs_ = inputs
		else:
			target_shape2 = min(inputs_shapes2)
			target_shape3 = min(inputs_shapes3)

			inputs_ = []
			for inp in inputs:
				diff2 = (inp.size(2) - target_shape2) // 2
				diff3 = (inp.size(3) - target_shape3) // 2
				inputs_.append(inp[:, :, diff2: diff2 + target_shape2, diff3:diff3 + target_shape3])

		return torch.cat(inputs_, dim=self.dim)

	def __len__(self):
		return len(self._modules)


class GenNoise(nn.Module):
	def __init__(self, dim2):
		super(GenNoise, self).__init__()
		self.dim2 = dim2

	def forward(self, input):
		a = list(input.size())
		a[1] = self.dim2
		# print (input.data.type())

		b = torch.zeros(a).type_as(input.data)
		b.normal_()

		x = torch.autograd.Variable(b)

		return x


class Swish(nn.Module):
	"""
		https://arxiv.org/abs/1710.05941
		The hype was so huge that I could not help but try it
	"""

	def __init__(self):
		super(Swish, self).__init__()
		self.s = nn.Sigmoid()

	def forward(self, x):
		return x * self.s(x)


def act(act_fun='LeakyReLU'):
	'''
		Either string defining an activation function or module (e.g. nn.ReLU)
	'''
	if isinstance(act_fun, str):
		if act_fun == 'LeakyReLU':
			return nn.LeakyReLU(0.2, inplace=True)
		elif act_fun == 'Swish':
			return Swish()
		elif act_fun == 'ELU':
			return nn.ELU()
		elif act_fun == 'none':
			return nn.Sequential()
		else:
			assert False
	else:
		return act_fun()


def bn(num_features):
	return nn.BatchNorm2d(num_features)


def conv(in_f, out_f, kernel_size, stride=1, bias=True, pad='zero', downsample_mode='stride'):
	downsampler = None
	if stride != 1 and downsample_mode != 'stride':

		if downsample_mode == 'avg':
			downsampler = nn.AvgPool2d(stride, stride)
		elif downsample_mode == 'max':
			downsampler = nn.MaxPool2d(stride, stride)
		elif downsample_mode in ['lanczos2', 'lanczos3']:
			downsampler = Downsampler(n_planes=out_f, factor=stride, kernel_type=downsample_mode, phase=0.5,
									  preserve_size=True)
		else:
			assert False

		stride = 1

	padder = None
	to_pad = int((kernel_size - 1) / 2)
	if pad == 'reflection':
		padder = nn.ReflectionPad2d(to_pad)
		to_pad = 0

	convolver = nn.Conv2d(in_f, out_f, kernel_size, stride, padding=to_pad, bias=bias)

	layers = filter(lambda x: x is not None, [padder, convolver, downsampler])
	return nn.Sequential(*layers)

def skip(
  num_input_channels=2, num_output_channels=3,
  num_channels_down=[16, 32, 64, 128, 128], num_channels_up=[16, 32, 64, 128, 128],
  num_channels_skip=[4, 4, 4, 4, 4],
  filter_size_down=3, filter_size_up=3, filter_skip_size=1,
  need_sigmoid=True, need_bias=True,
  pad='zero', upsample_mode='nearest', downsample_mode='stride', act_fun='LeakyReLU',
  need1x1_up=True):
  """Assembles encoder-decoder with skip connections.

  Arguments:
    act_fun: Either string 'LeakyReLU|Swish|ELU|none' or module (e.g. nn.ReLU)
    pad (string): zero|reflection (default: 'zero')
    upsample_mode (string): 'nearest|bilinear' (default: 'nearest')
    downsample_mode (string): 'stride|avg|max|lanczos2' (default: 'stride')

  """
  assert len(num_channels_down) == len(num_channels_up) == len(num_channels_skip)

  n_scales = len(num_channels_down)

  if not (isinstance(upsample_mode, list) or isinstance(upsample_mode, tuple)):
    upsample_mode = [upsample_mode] * n_scales

  if not (isinstance(downsample_mode, list) or isinstance(downsample_mode, tuple)):
    downsample_mode = [downsample_mode] * n_scales

  if not (isinstance(filter_size_down, list) or isinstance(filter_size_down, tuple)):
    filter_size_down = [filter_size_down] * n_scales

  if not (isinstance(filter_size_up, list) or isinstance(filter_size_up, tuple)):
    filter_size_up = [filter_size_up] * n_scales

  last_scale = n_scales - 1

  cur_depth = None

  model = nn.Sequential()
  model_tmp = model

  input_depth = num_input_channels
  for i in range(len(num_channels_down)):

    deeper = nn.Sequential()
    skip = nn.Sequential()

    if num_channels_skip[i] != 0:
      model_tmp.add(Concat(1, skip, deeper))
    else:
      model_tmp.add(deeper)

    model_tmp.add(bn(num_channels_skip[i] + (num_channels_up[i + 1] if i < last_scale else num_channels_down[i])))

    if num_channels_skip[i] != 0:
      skip.add(conv(input_depth, num_channels_skip[i], filter_skip_size, bias=need_bias, pad=pad))
      skip.add(bn(num_channels_skip[i]))
      skip.add(act(act_fun))

    # skip.add(Concat(2, GenNoise(nums_noise[i]), skip_part))

    deeper.add(conv(input_depth, num_channels_down[i], filter_size_down[i], 2, bias=need_bias, pad=pad,
            downsample_mode=downsample_mode[i]))
    deeper.add(bn(num_channels_down[i]))
    deeper.add(act(act_fun))

    deeper.add(conv(num_channels_down[i], num_channels_down[i], filter_size_down[i], bias=need_bias, pad=pad))
    deeper.add(bn(num_channels_down[i]))
    deeper.add(act(act_fun))

    deeper_main = nn.Sequential()

    if i == len(num_channels_down) - 1:
      # The deepest
      k = num_channels_down[i]
    else:
      deeper.add(deeper_main)
      k = num_channels_up[i + 1]

    deeper.add(nn.Upsample(scale_factor=2, mode=upsample_mode[i]))

    model_tmp.add(conv(num_channels_skip[i] + k, num_channels_up[i], filter_size_up[i], 1, bias=need_bias, pad=pad))
    model_tmp.add(bn(num_channels_up[i]))
    model_tmp.add(act(act_fun))

    if need1x1_up:
      model_tmp.add(conv(num_channels_up[i], num_channels_up[i], 1, bias=need_bias, pad=pad))
      model_tmp.add(bn(num_channels_up[i]))
      model_tmp.add(act(act_fun))

    input_depth = num_channels_down[i]
    model_tmp = deeper_main

  model.add(conv(num_channels_up[0], num_output_channels, 1, bias=need_bias, pad=pad))
  if need_sigmoid:
    model.add(nn.Sigmoid())

  return model


def get_net(input_depth, NET_TYPE, pad, upsample_mode, n_channels=3, act_fun='LeakyReLU', skip_n33d=128, skip_n33u=128,
			skip_n11=4, num_scales=5, downsample_mode='stride'):
	if NET_TYPE == 'skip':
		net = skip(input_depth, n_channels,
				   num_channels_down=[skip_n33d] * num_scales if isinstance(skip_n33d, int) else skip_n33d,
				   num_channels_up=[skip_n33u] * num_scales if isinstance(skip_n33u, int) else skip_n33u,
				   num_channels_skip=[skip_n11] * num_scales if isinstance(skip_n11, int) else skip_n11,
				   upsample_mode=upsample_mode, downsample_mode=downsample_mode,
				   need_sigmoid=True, need_bias=True, pad=pad, act_fun=act_fun)

	return net



In [7]:

global aug
global folder_name
global logit_idx
global RA_degrees
global RA_translate
global RandomGaussianBlur_kernelsize
global sharpness
global start_time
global noise_on
global cut_flag
global normalize_on
global input_noise


start_time = time.time()
sharpness = 0

#torch.use_deterministic_algorithms(True, warn_only=True)
seed = 1
np.random.seed(seed)
torch.manual_seed(seed)
random.seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

#sys.path.append("/home/mika/.conda/envs/py310/lib/python3.10/site-packages/torchvision/transforms/transforms.py")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def create_grid(batch_size, height, width):
    """ Creates a grid with a batch size of batch_size,
        2 channels, with given height and width """
    x = np.arange(width)
    y = np.arange(height)
    xv, yv = np.meshgrid(x, y)

    # create torch tensors from numpy ndarrays
    xv_t = torch.from_numpy(xv)
    yv_t = torch.from_numpy(yv)

    T = torch.stack((xv_t, yv_t))
    grid = T.repeat(batch_size, 1, 1, 1)
    return grid

def fill_noise(x, noise_type):
    """Fills tensor `x` with noise of type `noise_type`."""
    if noise_type == 'u':
        x.uniform_()
    elif noise_type == 'n':
        x.normal_()
    else:
        assert False


def get_noise(input_depth, method, spatial_size, noise_type='u', var=1. / 10):
    """Returns a pytorch.Tensor of size (1 x `input_depth` x `spatial_size[0]` x `spatial_size[1]`)
    initialized in a specific way.
    Args:
        input_depth: number of channels in the tensor
        method: `noise` for fillting tensor with noise; `meshgrid` for np.meshgrid
        spatial_size: spatial size of the tensor to initialize
        noise_type: 'u' for uniform; 'n' for normal
        var: a factor, a noise will be multiplicated by. Basically it is standard deviation scaler.
    """
    if isinstance(spatial_size, int):
        spatial_size = (spatial_size, spatial_size)
    if method == 'noise':
        shape = [torch.cuda.device_count(), input_depth, spatial_size[0], spatial_size[1]]
        net_input = torch.zeros(shape)

        fill_noise(net_input, noise_type)
        net_input *= var
    elif method == 'meshgrid':
        assert input_depth == 2
        X, Y = np.meshgrid(np.arange(0, spatial_size[1]) / float(spatial_size[1] - 1),
                           np.arange(0, spatial_size[0]) / float(spatial_size[0] - 1))
        meshgrid = np.concatenate([X[None, :], Y[None, :]])
        net_input = np_to_torch(meshgrid)
    else:
        assert False
    return net_input


def create_input_normal(input_depth, sideY, sideX, batch_size):
    # Create random input
    net_input = torch.zeros([batch_size, input_depth, sideY, sideX], device=device).normal_().div(10).detach()
    return net_input

class MakeCutouts(torch.nn.Module):
    def __init__(self, cut_size, cutn):
        global cut_flag
        super().__init__()
        self.cut_size = cut_size
        self.cutn = cutn
        self.cutflag = cut_flag
        # Compose image augmentations
        kernel_size = int(RandomGaussianBlur_kernelsize), int(RandomGaussianBlur_kernelsize)
        self.augs = torch.nn.Sequential(
            K.RandomAffine(degrees=RA_degrees, translate=RA_translate, p=0.8, padding_mode='border',
                           resample='bilinear'),
            K.RandomHorizontalFlip(p=0.5),
            K.RandomPerspective(0.45, p=0.8, resample='bilinear'),
            K.RandomGrayscale(p=0.15),
            K.RandomGaussianBlur(kernel_size, (0.1, 3)),
            K.RandomSharpness(sharpness=sharpness, p=0.5, keepdim=True),
            #K.RandomErasing(p=0.7)
        )

    def forward(self, input):
        sideY, sideX = input.shape[2:4]
        if sideY != sideX:
            input = K.RandomAffine(degrees=0, shear=10, p=0.5)(input)
            # if sizes are different, shear the image, so it fits into a square

        cutouts = []
        cn_size = [self.cut_size, self.cut_size]

        if self.cutflag:
            max_size = min(sideX, sideY)
            for cn in range(self.cutn):
                if cn > self.cutn - self.cutn // 4:
                    cutout = input
                else:
                    size = int(
                        max_size * torch.zeros(1, ).normal_(mean=.8, std=.3).clip(float(self.cut_size / max_size), 1.))
                    offsetx = torch.randint(0, sideX - size + 1, ())
                    offsety = torch.randint(0, sideY - size + 1, ())
                    cutout = input[:, :, offsety:offsety + size, offsetx:offsetx + size]
                cutout = F.adaptive_avg_pool2d(cutout, self.cut_size)
                cutouts.append(cutout)

        else:
            for cn in range(self.cutn):
                cutout = input
                cutout = F.avg_pool2d(cutout, kernel_size=4, stride=2, padding=1)
                cutouts.append(cutout)

        cutouts = torch.cat(cutouts)  # Concatenate cutouts - the rows
        cutouts = self.augs(cutouts)
        return cutouts


def optimize_network(num_iterations, seed, input_depth, input_dims, optimizer_type, lr,
                     lower_lr, display_rate, cut_size, cutn, results_folder, model, name=None):

    loss_list = []

    if seed is not None:
        np.random.seed(seed)
        torch.manual_seed(seed)
        random.seed(seed)
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)

    # create results folder
    if not os.path.isdir(results_folder):
        os.makedirs(results_folder)

    """
    # create results sub folder
    full_sub_folder = results_folder + "/" + folder_name
    if not os.path.isdir(full_sub_folder):
        os.makedirs(full_sub_folder)
    """

    make_cutouts = MakeCutouts(cut_size, cutn)
    make_cutouts = nn.DataParallel(make_cutouts)

    # Initialize DIP skip network
    DIP_net = get_net(
        input_depth, 'skip',
        pad='reflection',
        skip_n33d=128, skip_n33u=128,
        skip_n11=4, num_scales=7,
        upsample_mode='bilinear',
    ).to(device)

    DIP_net = nn.DataParallel(DIP_net)

    # Initialize input noise
    input_depth = input_dims[0]
    sideY = input_dims[1]
    sideX = input_dims[2]
    # net_input = create_input_normal(input_depth, sideY, sideX, batch_size)

    if input_noise:
        net_input = get_noise(input_depth, 'noise', (sideY, sideX), noise_type='u', var=1. / 10)

    else: # input is positional encoding
        net_input = create_grid(batch_size= torch.cuda.device_count(), height = sideY, width = sideX)

    net_input = net_input.type(torch.cuda.FloatTensor)
    net_input.to(device)

    if optimizer_type == 'Adam':
        optimizer = torch.optim.Adam(DIP_net.parameters(), lr)
    elif optimizer_type == 'MADGRAD':
        optimizer = MADGRAD(DIP_net.parameters(), lr, weight_decay=0.01, momentum=0.9)

    # get model
    if model == "VIT_B_16":
        classifier = torchvision.models.vit_b_16(
            weights=torchvision.models.vision_transformer.ViT_B_16_Weights.IMAGENET1K_V1)
        mean = [0.485, 0.456, 0.406]
        std = [0.229, 0.224, 0.225]
    if model == "EFFICIENTNET":
        classifier = torchvision.models.efficientnet_v2_l(
            weights=torchvision.models.EfficientNet_V2_L_Weights.IMAGENET1K_V1)
        mean = [0.5, 0.5, 0.5]
        std = [0.5, 0.5, 0.5]

    classifier.to(device)
    classifier.eval()
    classifier = nn.DataParallel(classifier)

    # training
    #    try:
    for i in range(num_iterations):
        optimizer.zero_grad(set_to_none=True)
        if noise_on:
            if input_noise:
                noise = get_noise(input_depth, 'noise', (sideY, sideX), noise_type='n', var=0.01)
                # noise = noise.type(torch.cuda.FloatTensor)
            else: # positional encoding
                noise = get_noise(input_depth, 'noise', (sideY, sideX), noise_type='u', var=1.0)

            noise = noise.to(device=device)
            noisy_net_input = net_input + noise

            with torch.cuda.amp.autocast():  # ops run in an op-specific dtype chosen by autocast to improve performance
                dip_out = DIP_net(noisy_net_input).float()

        else:
            with torch.cuda.amp.autocast():  # ops run in an op-specific dtype chosen by autocast to improve performance
                dip_out = DIP_net(net_input).float()  # run net on input

        cutouts = make_cutouts(dip_out)

        if normalize_on:
            normalize_func = kornia.enhance.Normalize(mean, std)
            cutouts = normalize_func(cutouts)

        out = classifier(cutouts)
        out = F.softmax(out, dim=1)
        loss = 1 - out[:, logit_idx].mean()

        loss_list.append(loss)
        loss.backward()
        optimizer.step()

        if ((i+1) % display_rate) == 0:
            print("inside if")
            image = TF.to_pil_image(dip_out[0])
            path = results_folder + "/" + f'{name}_res.png'
            print(path)
            image.save(path, quality=100)

        if lower_lr:  # lower the learning rate over time - multiply by 0.99
            optimizer.param_groups[0]['lr'] = max(0.00001, .99 * optimizer.param_groups[0]['lr'])

        print(f'Iteration {i} of {num_iterations}')

    # save DIP weights
    #torch.save(DIP_net.state_dict(), full_sub_folder + "/net")
    return dip_out[0], loss_list


def train_net(results_folder, name=None):
    global input_noise
    global combine_models

    num_iterations = 1000 #2500
    seed = 1  # random.randint(0, 2 ** 32)
    sideY = 512  # 512
    sideX = 512  # 512
    # logit_idx = 1 became global
    optimizer_type = 'Adam'
    lr = 1e-3
    lower_lr = False
    display_rate = 1000
    cut_size = 224
    cutn = 11
    model = "EFFICIENTNET" #"VIT_H_14" # "EFFICIENTNET" #"VIT_B_16"
    input_noise = 1  # determines if input is noise or positional encoding

    if input_noise:
        input_depth = 32
    else: # positional encoding
        input_depth = 2

    input_dims = (input_depth, sideY, sideX)

    out, loss_list = optimize_network(num_iterations, seed, input_depth, input_dims, optimizer_type, lr,
                                      lower_lr, display_rate, cut_size, cutn, results_folder, model, name)

    #create_config_file(num_iterations, seed, input_depth, sideY, sideX, optimizer_type, lr, lower_lr, cut_size, cutn,
    #                   results_folder)
    #create_loss_file(loss_list, results_folder)
    #create_loss_graph(loss_list, results_folder)
    #create_smooth_loss_graph(loss_list,results_folder)
    #create_log_loss_graph(loss_list, results_folder)

    return out


def create_loss_file(loss_list, results_folder):
    path = results_folder + "/" + folder_name + "/Loss.txt"
    file = open(path, "a")
    for i, loss in enumerate(loss_list):
        text = f'Iterarion {i}: loss = {loss}\n'
        file.write(text)

    file.close()


def create_loss_graph(loss_list, results_folder):
    path = results_folder + "/" + folder_name + "/Loss.png"
    iterations = np.array(range(1, len(loss_list) + 1))
    loss = np.array(loss_list)
    plt.plot(iterations, loss, color='r', label='Loss')
    plt.xlabel("Iteration")
    plt.ylabel("Loss")
    plt.title("The Loss as a Function of the iteration number")
    plt.savefig(path)

def create_smooth_loss_graph(loss_list,results_folder):
    path = results_folder + "/" + folder_name + "/Loss_smoothed.png"
    loss = torch.mean(torch.Tensor(loss_list).view(-1, 50), dim=1)
    plt.plot(range(1, len(loss_list)//50 + 1), loss, color='r', label='Loss')
    plt.xlabel("Iteration")
    plt.ylabel("Smooth Loss")
    plt.title("The 5 timestep mean Loss as a Function of the iteration number")
    plt.savefig(path)

def create_log_loss_graph(loss_list, results_folder):
    path = results_folder + "/" + folder_name + "/Log_Loss.png"
    iterations = np.array(range(1, len(loss_list) + 1))
    log = np.log(np.array(loss_list))
    plt.plot(iterations, log, color='r', label='log(Loss)')
    plt.xlabel("Iteration")
    plt.ylabel("Log(Loss)")
    plt.title("The Logarithm of the Loss as a Function of the iteration number")
    plt.savefig(path)

def get_config(sharp_on, logit):
    """
    assigns configuration
    parameters to global variables
    """
    global folder_name
    global logit_idx
    global RA_degrees
    global RA_translate
    global RandomGaussianBlur_kernelsize
    global sharpness
    global noise_on
    global cut_flag
    global normalize_on

    folder_name = "results"
    logit_idx = logit
    RA_degrees = 30
    RA_translate = 0.3
    RandomGaussianBlur_kernelsize = 31
    noise_on = 0
    cut_flag = 1
    normalize_on = 0
    if sharp_on:
        sharpness = 0

def run_config(results_folder, sharp_on, logit, name=None):
    """
    runs a config
    """
    get_config(sharp_on, logit)
    out = train_net(results_folder, name)
    return out

sharp_on = False
results_folder_base = "OUTPUTS"


# Run Code

In [10]:
results_folder = results_folder_base + class_name
run_config(results_folder, True, class_number, name=None)

Downloading: "https://download.pytorch.org/models/efficientnet_v2_l-59c71312.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_v2_l-59c71312.pth
100%|██████████| 455M/455M [00:05<00:00, 94.7MB/s]


Iteration 0 of 1000
Iteration 1 of 1000
Iteration 2 of 1000
Iteration 3 of 1000
Iteration 4 of 1000
Iteration 5 of 1000
Iteration 6 of 1000
Iteration 7 of 1000
Iteration 8 of 1000


KeyboardInterrupt: ignored